In [2]:
!pip install -q groq huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 10.1 MB/s eta 0:00:00


In [3]:
import os
import pandas as pd
from groq import Groq
import json
import time
from sklearn.metrics import classification_report
import pandas as pd
GROQ_API_KEY = "" # Paste your Groq API key here

groq_client = Groq(api_key=GROQ_API_KEY)

models = [
    "llama-3.1-8b-instant",
    "llama-3.3-70b-versatile"
]

print(f"Let's compare: {models}")
valid_labels = {"joy", "sadness", "anger", "fear", "anticipation"}

Let's compare: ['llama-3.1-8b-instant', 'llama-3.3-70b-versatile']


In [4]:
df_full = pd.read_csv("val.csv")

def get_one_per_class(group):
    return group.sample(n=1, random_state=42)

eval_df = (
    df_full
    .groupby(['lang', 'label'], group_keys=False)
    .apply(get_one_per_class)
)

eval_df = eval_df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"Test size: {len(eval_df)}")

print(eval_df.groupby('lang')['label'].value_counts())


Test size: 89
lang  label       
bn    anger           1
      anticipation    1
      fear            1
      joy             1
      sadness         1
                     ..
zh    anger           1
      anticipation    1
      fear            1
      joy             1
      sadness         1
Name: count, Length: 89, dtype: int64


/tmp/ipython-input-591846657.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(get_one_per_class)


In [5]:
import re

def clean_text(text):
    text = str(text)
    text = re.sub(r'\[[A-Z]+\]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


eval_df['text'] = eval_df['text'].apply(clean_text)

eval_df = eval_df[eval_df['text'] != '']


In [6]:
hi_fear = eval_df[
    (eval_df['lang'] == 'hi') &
    (eval_df['label'] == 'fear')
]

if not hi_fear.empty:
    print("Hindi + Fear found:")
    print(hi_fear['text'].values[0])
else:
    print("Hindi + Fear not found.")


Hindi + Fear found:
वडोदरावासियों को अलर्ट रहने को कहा है... वडोदरा महानगर पालिका (Vadodara municipal corporation) ने। Source : Sandesh news. Be Careful Vadodara ma Ek mahila pani jova Gai hati ane maggar aene pani ma khenchi Gayo Che to pani jova na jaso vadodara ma maggar bahu j che. with Jigar Patel and 00...


In [24]:
def get_system_prompt(config_type):
    valid_labels = "joy, sadness, anger, fear, anticipation"

    if config_type == "zero-shot":
        return (
            f"You are an expert linguist specialized in emotion detection. "
            f"Classify the emotion of the text into exactly one of these labels: {valid_labels}. "
            f"Return ONLY JSON in the format {{\"emotion\": \"label\"}}."
        )

    elif config_type == "few-shot":
        return (
            "You are an expert in multilingual emotion classification.\n"
            "Possible labels: joy, sadness, anger, fear, anticipation.\n"
            "Use ONLY these labels.\n\n"

            "Examples:\n"
            "Text: I finally got the job I wanted!\nEmotion: joy\n\n"
            "Text: I lost my wallet yesterday.\nEmotion: sadness\n\n"
            "Text: This is so unfair, I'm furious.\nEmotion: anger\n\n"
            "Text: I don't know what will happen tomorrow.\nEmotion: anticipation\n\n"
            "Text: Mujhe andhere se dar lagta hai.\nEmotion: fear\n\n"

            "Now classify the following text.\n"
            "Return ONLY JSON in the format {\"emotion\": \"label\"}"
        )

    elif config_type == "chain-of-thought":
        return (
            "You are an expert multilingual linguist and psychologist. "
            "Your task is to analyze the emotional state of a social media post.\n\n"
            "Follow these steps:\n"
            "1. Identify the language of the text.\n"
            "2. Briefly analyze key emotional keywords and context.\n"
            "3. Choose the most fitting emotion from this list: [joy, sadness, anger, fear, anticipation].\n\n"
            "Output your reasoning and the final label in JSON format like this:\n"
            "{\n"
            "  \"reasoning\": \"Step-by-step analysis...\",\n"
            "  \"emotion\": \"label\"\n"
            "}"
        )


In [25]:
def normalize_prediction(pred):
    if pred is None:
        return "error"

    pred = pred.lower().strip()
    return pred if pred in valid_labels else "error"

In [26]:
import re

def predict_emotion(text, model_name, config_type):
    system_content = get_system_prompt(config_type)
    user_content = f"Text: {text}"
    try:
        response = groq_client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": system_content},
                {"role": "user", "content": user_content}
            ],
            temperature=0.0
        )
        result = response.choices[0].message.content

        try:
            start = result.find('{')
            end = result.rfind('}') + 1
            if start != -1 and end != -1:
                json_str = result[start:end]
                clean_json_str = json_str.replace('\n', ' ').replace('\r', '')
                raw_pred = json.loads(clean_json_str).get("emotion", None)
                return normalize_prediction(raw_pred)
        except:
            pass

        match = re.search(r'"emotion":\s*"([^"]+)"', result, re.IGNORECASE)
        if match:
            return normalize_prediction(match.group(1))

        return "error"

    except Exception as e:
        print(f"Error with {model_name}: {e}")
        time.sleep(1)
        return "error"


In [14]:
results = []

models = [
    "llama-3.1-8b-instant",
    "llama-3.3-70b-versatile"
]

prompt_config = "few-shot"

print(f"Starting experiment: {prompt_config}\n")

for _, row in eval_df.iterrows():
    text = row['text']
    true_label = row['label']

    print(f"Processing: {text[:40]}...")

    for model in models:
        pred = predict_emotion(text, model, prompt_config)

        results.append({
            "text": text,
            "language": row['lang'],
            "true_label": true_label,
            "model": model,
            "config": prompt_config,
            "predicted": pred
        })

res_df = pd.DataFrame(results)
display(res_df)


Starting experiment: few-shot

Processing: កុំជោ ជាមួយស្រុកខ្មែរ ទោះជិះម៉ូតូទំនើបក៏...
Processing: အေမ့ ေျခရာ ....
Processing: Penak e seng oleh tiket final batle. Mar...
Processing: willing to adopt/foster him...
Processing: Ogy Ziana Pesawat MAS MH00 terhempas di ...
Processing: Mejaj kharap thakar upojukto karon মেজাজ...
Processing: [ ] Que prova estamos passando cada dia ...
Processing: Hoy fue el cumple de mi y subi a darle s...
Processing: everything... Anyway Hope you all have a...
Processing: venait de décider de les adopter tous le...
Processing: যতখন আমি আমার ইউনিভারসিটিতে বন্ধুদের সাথ...
Processing: Salam sahabat0 semua... jom amalkan sura...
Processing: ុរសម្នាក់ថានឹងមករៀបការជាមួយនាងយកទៅក្រៅប្...
Processing: ” ============================ Có người ...
Processing: dang dở :'( Mong là sống sót sau con tră...
Processing: figlcome te ricordo dal primo giorno che...
Processing: .. hinihintay mo maubos ung tubig sa tab...
Processing: pensamento: “Que nesse ano sejamos, Todo...
Pr

,text,language,true_label,model,config,predicted
0,កុំជោ ជាមួយស្រុកខ្មែរ ទោះជិះម៉ូតូទំនើបក៏មិនឡូយ...,km,sadness,llama-3.1-8b-instant,few-shot,sadness
1,កុំជោ ជាមួយស្រុកខ្មែរ ទោះជិះម៉ូតូទំនើបក៏មិនឡូយ...,km,sadness,llama-3.3-70b-versatile,few-shot,sadness
2,အေမ့ ေျခရာ .,my,joy,llama-3.1-8b-instant,few-shot,sadness
3,အေမ့ ေျခရာ .,my,joy,llama-3.3-70b-versatile,few-shot,sadness
4,Penak e seng oleh tiket final batle. Marai png...,id,anger,llama-3.1-8b-instant,few-shot,fear
...,...,...,...,...,...,...
173,ร้อยก็ไม่มี],th,fear,llama-3.3-70b-versatile,few-shot,sadness
174,"please everyone her whole life, but at the end...",en,sadness,llama-3.1-8b-instant,few-shot,sadness
175,"please everyone her whole life, but at the end...",en,sadness,llama-3.3-70b-versatile,few-shot,sadness
176,းပါရေစ....... 0000 ထိ အသာေလး ဆက္ အရွံဳးခံလိုက္...,my,anticipation,llama-3.1-8b-instant,few-shot,fear


In [15]:
def calculate_accuracy(df, model_name):
    subset = df[df['model'] == model_name]
    correct = subset[
        subset['predicted'] == subset['true_label']
    ]
    return len(correct) / len(subset)

In [16]:
label_order = ["joy", "sadness", "anger", "fear", "anticipation"]

for model in models:
    print(f"Detailed Classification Report ({model})")

    model_data = res_df[res_df['model'] == model]

    print(classification_report(
        model_data['true_label'],
        model_data['predicted'],
        labels=label_order,
        zero_division=0
    ))


Detailed Classification Report (llama-3.1-8b-instant)
              precision    recall  f1-score   support

         joy       0.34      0.67      0.45        18
     sadness       0.45      0.56      0.50        18
       anger       0.62      0.28      0.38        18
        fear       0.60      0.53      0.56        17
anticipation       0.50      0.22      0.31        18

   micro avg       0.45      0.45      0.45        89
   macro avg       0.50      0.45      0.44        89
weighted avg       0.50      0.45      0.44        89

Detailed Classification Report (llama-3.3-70b-versatile)
              precision    recall  f1-score   support

         joy       0.45      0.78      0.57        18
     sadness       0.64      0.78      0.70        18
       anger       0.60      0.33      0.43        18
        fear       1.00      0.47      0.64        17
anticipation       0.50      0.50      0.50        18

    accuracy                           0.57        89
   macro avg       0

In [17]:
results = []

models = [
    "llama-3.1-8b-instant",
    "llama-3.3-70b-versatile"
]

prompt_config = "zero-shot"

print(f"Starting experiment: {prompt_config}\n")

for _, row in eval_df.iterrows():
    text = row['text']
    true_label = row['label']

    print(f"Processing: {text[:40]}...")

    for model in models:
        pred = predict_emotion(text, model, prompt_config)

        results.append({
            "text": text,
            "language": row['lang'],
            "true_label": true_label,
            "model": model,
            "config": prompt_config,
            "predicted": pred
        })

res_df1 = pd.DataFrame(results)
display(res_df1)

Starting experiment: zero-shot

Processing: កុំជោ ជាមួយស្រុកខ្មែរ ទោះជិះម៉ូតូទំនើបក៏...
Processing: အေမ့ ေျခရာ ....
Processing: Penak e seng oleh tiket final batle. Mar...
Processing: willing to adopt/foster him...
Processing: Ogy Ziana Pesawat MAS MH00 terhempas di ...
Processing: Mejaj kharap thakar upojukto karon মেজাজ...
Processing: [ ] Que prova estamos passando cada dia ...
Processing: Hoy fue el cumple de mi y subi a darle s...
Processing: everything... Anyway Hope you all have a...
Processing: venait de décider de les adopter tous le...
Processing: যতখন আমি আমার ইউনিভারসিটিতে বন্ধুদের সাথ...
Processing: Salam sahabat0 semua... jom amalkan sura...
Processing: ុរសម្នាក់ថានឹងមករៀបការជាមួយនាងយកទៅក្រៅប្...
Processing: ” ============================ Có người ...
Processing: dang dở :'( Mong là sống sót sau con tră...
Processing: figlcome te ricordo dal primo giorno che...
Processing: .. hinihintay mo maubos ung tubig sa tab...
Processing: pensamento: “Que nesse ano sejamos, Todo...
P

,text,language,true_label,model,config,predicted
0,កុំជោ ជាមួយស្រុកខ្មែរ ទោះជិះម៉ូតូទំនើបក៏មិនឡូយ...,km,sadness,llama-3.1-8b-instant,zero-shot,sadness
1,កុំជោ ជាមួយស្រុកខ្មែរ ទោះជិះម៉ូតូទំនើបក៏មិនឡូយ...,km,sadness,llama-3.3-70b-versatile,zero-shot,sadness
2,အေမ့ ေျခရာ .,my,joy,llama-3.1-8b-instant,zero-shot,sadness
3,အေမ့ ေျခရာ .,my,joy,llama-3.3-70b-versatile,zero-shot,sadness
4,Penak e seng oleh tiket final batle. Marai png...,id,anger,llama-3.1-8b-instant,zero-shot,sadness
...,...,...,...,...,...,...
173,ร้อยก็ไม่มี],th,fear,llama-3.3-70b-versatile,zero-shot,sadness
174,"please everyone her whole life, but at the end...",en,sadness,llama-3.1-8b-instant,zero-shot,sadness
175,"please everyone her whole life, but at the end...",en,sadness,llama-3.3-70b-versatile,zero-shot,sadness
176,းပါရေစ....... 0000 ထိ အသာေလး ဆက္ အရွံဳးခံလိုက္...,my,anticipation,llama-3.1-8b-instant,zero-shot,sadness


In [18]:
label_order = ["joy", "sadness", "anger", "fear", "anticipation"]

for model in models:
    print(f"Detailed Classification Report ({model})")

    model_data = res_df1[res_df1['model'] == model]

    print(classification_report(
        model_data['true_label'],
        model_data['predicted'],
        labels=label_order,
        zero_division=0
    ))

Detailed Classification Report (llama-3.1-8b-instant)
              precision    recall  f1-score   support

         joy       0.35      0.72      0.47        18
     sadness       0.39      0.67      0.49        18
       anger       0.67      0.33      0.44        18
        fear       0.78      0.41      0.54        17
anticipation       1.00      0.11      0.20        18

   micro avg       0.45      0.45      0.45        89
   macro avg       0.64      0.45      0.43        89
weighted avg       0.63      0.45      0.43        89

Detailed Classification Report (llama-3.3-70b-versatile)
              precision    recall  f1-score   support

         joy       0.45      0.78      0.57        18
     sadness       0.56      0.78      0.65        18
       anger       0.50      0.28      0.36        18
        fear       1.00      0.29      0.45        17
anticipation       0.50      0.50      0.50        18

    accuracy                           0.53        89
   macro avg       0

In [29]:
results = []

models = [
    "llama-3.1-8b-instant",
    "llama-3.3-70b-versatile"
]

prompt_config = "chain-of-thought"

print(f"Starting experiment: {prompt_config}\n")

for _, row in eval_df.iterrows():
    text = row['text']
    true_label = row['label']

    print(f"Processing: {text[:40]}...")

    for model in models:
        pred = predict_emotion(text, model, prompt_config)

        results.append({
            "text": text,
            "language": row['lang'],
            "true_label": true_label,
            "model": model,
            "config": prompt_config,
            "predicted": pred
        })

res_df2 = pd.DataFrame(results)
display(res_df2)

Starting experiment: chain-of-thought

Processing: កុំជោ ជាមួយស្រុកខ្មែរ ទោះជិះម៉ូតូទំនើបក៏...
Processing: အေမ့ ေျခရာ ....
Processing: Penak e seng oleh tiket final batle. Mar...
Processing: willing to adopt/foster him...
Processing: Ogy Ziana Pesawat MAS MH00 terhempas di ...
Processing: Mejaj kharap thakar upojukto karon মেজাজ...
Processing: [ ] Que prova estamos passando cada dia ...
Processing: Hoy fue el cumple de mi y subi a darle s...
Processing: everything... Anyway Hope you all have a...
Processing: venait de décider de les adopter tous le...
Processing: যতখন আমি আমার ইউনিভারসিটিতে বন্ধুদের সাথ...
Processing: Salam sahabat0 semua... jom amalkan sura...
Processing: ុរសម្នាក់ថានឹងមករៀបការជាមួយនាងយកទៅក្រៅប្...
Processing: ” ============================ Có người ...
Processing: dang dở :'( Mong là sống sót sau con tră...
Processing: figlcome te ricordo dal primo giorno che...
Processing: .. hinihintay mo maubos ung tubig sa tab...
Processing: pensamento: “Que nesse ano sejamos, To

,text,language,true_label,model,config,predicted
0,កុំជោ ជាមួយស្រុកខ្មែរ ទោះជិះម៉ូតូទំនើបក៏មិនឡូយ...,km,sadness,llama-3.1-8b-instant,chain-of-thought,sadness
1,កុំជោ ជាមួយស្រុកខ្មែរ ទោះជិះម៉ូតូទំនើបក៏មិនឡូយ...,km,sadness,llama-3.3-70b-versatile,chain-of-thought,sadness
2,အေမ့ ေျခရာ .,my,joy,llama-3.1-8b-instant,chain-of-thought,sadness
3,အေမ့ ေျခရာ .,my,joy,llama-3.3-70b-versatile,chain-of-thought,sadness
4,Penak e seng oleh tiket final batle. Marai png...,id,anger,llama-3.1-8b-instant,chain-of-thought,sadness
...,...,...,...,...,...,...
173,ร้อยก็ไม่มี],th,fear,llama-3.3-70b-versatile,chain-of-thought,sadness
174,"please everyone her whole life, but at the end...",en,sadness,llama-3.1-8b-instant,chain-of-thought,sadness
175,"please everyone her whole life, but at the end...",en,sadness,llama-3.3-70b-versatile,chain-of-thought,sadness
176,းပါရေစ....... 0000 ထိ အသာေလး ဆက္ အရွံဳးခံလိုက္...,my,anticipation,llama-3.1-8b-instant,chain-of-thought,sadness


In [30]:
label_order = ["joy", "sadness", "anger", "fear", "anticipation"]

for model in models:
    print(f"Detailed Classification Report ({model})")

    model_data = res_df2[res_df2['model'] == model]

    print(classification_report(
        model_data['true_label'],
        model_data['predicted'],
        labels=label_order,
        zero_division=0
    ))

Detailed Classification Report (llama-3.1-8b-instant)
              precision    recall  f1-score   support

         joy       0.44      0.67      0.53        18
     sadness       0.40      0.56      0.47        18
       anger       0.75      0.50      0.60        18
        fear       0.88      0.41      0.56        17
anticipation       0.53      0.44      0.48        18

   micro avg       0.53      0.52      0.52        89
   macro avg       0.60      0.52      0.53        89
weighted avg       0.60      0.52      0.53        89

Detailed Classification Report (llama-3.3-70b-versatile)
              precision    recall  f1-score   support

         joy       0.44      0.83      0.58        18
     sadness       0.55      0.67      0.60        18
       anger       0.64      0.39      0.48        18
        fear       1.00      0.41      0.58        17
anticipation       0.69      0.50      0.58        18

   micro avg       0.57      0.56      0.57        89
   macro avg       0

In [7]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import classification_report
from tqdm import tqdm


In [11]:
from google.colab import drive
drive.mount('/content/drive')

!unzip /content/drive/MyDrive/results.zip -d my_saved_model

Mounted at /content/drive
Archive:  /content/drive/MyDrive/results.zip
   creating: my_saved_model/results/checkpoint-7620/
  inflating: my_saved_model/results/checkpoint-7620/config.json  
  inflating: my_saved_model/results/checkpoint-7620/model.safetensors  
  inflating: my_saved_model/results/checkpoint-7620/optimizer.pt  
  inflating: my_saved_model/results/checkpoint-7620/scheduler.pt  
  inflating: my_saved_model/results/checkpoint-7620/rng_state.pth  
  inflating: my_saved_model/results/checkpoint-7620/training_args.bin  
  inflating: my_saved_model/results/checkpoint-7620/trainer_state.json  
  inflating: my_saved_model/results/checkpoint-7620/scaler.pt  


In [12]:
MODEL_PATH = "my_saved_model/results/checkpoint-7620"
BASE_MODEL_NAME = "microsoft/mdeberta-v3-base"

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Loading model from {MODEL_PATH} to {device}...")

Loading model from my_saved_model/results/checkpoint-7620 to cpu...


In [14]:
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
except:
    print(f"Tokenizer not found in checkpoint. Loading base tokenizer: {BASE_MODEL_NAME}")
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)

model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)
model.to(device)
model.eval()

Tokenizer not found in checkpoint. Loading base tokenizer: microsoft/mdeberta-v3-base


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(251000, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.3, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): Dropout(p=0.3, inplace=False)
              (dropout): Dropout(p=0.3, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

In [15]:
print("Labels in model:", model.config.id2label)

Labels in model: {0: 'LABEL_0', 1: 'LABEL_1', 2: 'LABEL_2', 3: 'LABEL_3', 4: 'LABEL_4'}


In [19]:
my_id2label = {
    0: "anger",
    1: "anticipation",
    2: "fear",
    3: "joy",
    4: "sadness"
}
model.config.id2label = my_id2label

print("Fixed Labels:", model.config.id2label)

Fixed Labels: {0: 'anger', 1: 'anticipation', 2: 'fear', 3: 'joy', 4: 'sadness'}


In [20]:
def predict_batch(texts, batch_size=16):
    all_preds = []

    for i in tqdm(range(0, len(texts), batch_size), desc="Predicting"):
        batch_texts = texts[i : i + batch_size]

        inputs = tokenizer(
            batch_texts,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=128
        ).to(device)

        with torch.no_grad():
            logits = model(**inputs).logits

        preds = torch.argmax(logits, dim=1).cpu().numpy()

        pred_labels = [model.config.id2label[p_id] for p_id in preds]
        all_preds.extend(pred_labels)

    return all_preds

In [22]:
texts_to_predict = eval_df['text'].tolist()
y_pred_finetuned = predict_batch(texts_to_predict)
eval_df['pred_finetuned'] = y_pred_finetuned

Predicting: 100%|██████████| 6/6 [00:37<00:00,  6.31s/it]


In [23]:
print("Classification Report for Fine-Tuned Model")
print(classification_report(
    eval_df['label'],
    eval_df['pred_finetuned'],
    zero_division=0
))

Classification Report for Fine-Tuned Model
              precision    recall  f1-score   support

       anger       0.69      0.50      0.58        18
anticipation       0.50      0.50      0.50        18
        fear       1.00      0.24      0.38        17
         joy       0.50      0.67      0.57        18
     sadness       0.47      0.78      0.58        18

    accuracy                           0.54        89
   macro avg       0.63      0.54      0.52        89
weighted avg       0.63      0.54      0.52        89

